In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install gradio


In [ ]:
from transformers import pipeline
import re
import pandas as pd
import gradio as gr


In [ ]:
sentiment_model = pipeline("sentiment-analysis")


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text


In [ ]:
def classify_category(text):
    categories = {
        "Roads": ["pothole", "road", "street", "manhole", "bump", "footpath", "sidewalk"],
        "Water Supply & Drainage": ["water", "leak", "pipe", "tap", "sewage", "drain", "overflow", "blockage"],
        "Electricity": ["electricity", "power", "voltage", "wire", "short circuit", "transformer", "light", "pole"],
        "Garbage/Sanitation": ["garbage", "trash", "waste", "dump", "bin", "cleaning", "sanitation"],
        "Animal Control": ["dog", "monkey", "animal", "stray", "bite", "dead dog", "cow"],
        "Noise Pollution": ["noise", "loud", "speaker", "dj", "disturbance", "music"],
    }

    for category, keywords in categories.items():
        for keyword in keywords:
            if keyword in text:
                return category
    return "Not a municipal issue"


In [ ]:
def determine_urgency(sentiment_result, text, category):
    emergency_keywords = [
        "accident", "urgent", "immediate", "emergency", "short circuit",
        "dead", "injury", "overflow", "sewage", "fire", "open wire", "live wire", "no water", "road block"
    ]
    medium_keywords = [
        "delayed", "pending", "not picked", "not fixed",
        "leak", "damaged", "complaint", "no response", "dirty"
    ]
    low_keywords = ["dim", "dust", "old", "inconvenience", "not working"]

    if category == "Not a municipal issue":
        return "Not applicable"

    if any(word in text for word in emergency_keywords):
        return "High"
    elif any(word in text for word in medium_keywords):
        return "Medium"
    elif any(word in text for word in low_keywords):
        return "Low"

    label = sentiment_result[0]['label']
    score = sentiment_result[0]['score']
    if label == "NEGATIVE" and score > 0.8:
        return "Medium"
    else:
        return "Low"


In [ ]:
complaint_log = []


In [ ]:
def analyze_complaint_gradio_log(text):
    cleaned = preprocess_text(text)
    sentiment_result = sentiment_model(cleaned)
    category = classify_category(cleaned)
    urgency = determine_urgency(sentiment_result, cleaned, category)

    complaint_log.append({
        "Complaint": text,
        "Category": category,
        "Urgency": urgency,
        "Sentiment": sentiment_result[0]['label'],
        "Confidence": round(sentiment_result[0]['score'], 2)
    })

    result = f"📝 Complaint: {text}\n\n📂 Category: {category}\n\n🚨 Urgency Level: {urgency}\n\n🧠 Sentiment: {sentiment_result[0]['label']} (Confidence: {sentiment_result[0]['score']:.2f})"
    return result


In [ ]:
interface = gr.Interface(
    fn=analyze_complaint_gradio_log,
    inputs=gr.Textbox(lines=4, placeholder="Type your complaint here..."),
    outputs="text",
    title="Municipal Complaint Classifier",
    description="AI-based system to classify public complaints and assess urgency for municipal response."
)

interface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://56218bb5ad5ab9b5ed.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# Show log as a table
df = pd.DataFrame(complaint_log)
df


,Complaint,Category,Urgency,Sentiment,Confidence
0,my fan is not working,Not a municipal issue,Not applicable,NEGATIVE,1.00
1,no water,Water Supply & Drainage,High,NEGATIVE,1.00
2,loud music,Noise Pollution,Low,POSITIVE,0.87
3,my tap keeps on leaking,Water Supply & Drainage,Medium,NEGATIVE,1.00
4,there are many dogs in the area and residents ...,Animal Control,Medium,NEGATIVE,0.95
5,I am getting dirty smell from the road,Roads,Medium,NEGATIVE,1.00
6,the water supply is good,Water Supply & Drainage,Low,POSITIVE,1.00
7,there is no electricity,Electricity,Medium,NEGATIVE,1.00
8,A transformer is giving sparks,Electricity,Low,POSITIVE,1.00
